In [3]:
!pip install iterative-stratification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 345.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 414.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip install -U accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
!pip install torch transformers datasets sentencepiece safetensors numpy pandas protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 298.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 250.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 348.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 156.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 379.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 228.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 255.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 381.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 449.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 253.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
# 1) 환경/경로 설정

import os, json, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

DATA_PATH = "hf_perfumes_mapped_ko_clean_stopwords_saved.csv"   # <- 필요시 경로 수정
TEXT_COL  = "description_clean"                  # 영어 문장
LABEL_COL = "main_accord_ko"               # 한글 라벨(공백 구분)
OUTPUT_DIR = "./hf_xlmr_ckpt"              # 모델 저장 폴더
MODEL_NAME = "xlm-roberta-base"            # 다국어 베이스


# 2) 데이터 로드 + 결측/빈 라벨 처리

# 로드
df = pd.read_csv(DATA_PATH, encoding="utf-8-sig")
df[LABEL_COL] = df[LABEL_COL].fillna("").astype(str)

# 공백 구분 다중라벨 -> 리스트
def split_labels(s: str):
    return [t for t in s.split() if t.strip()]

df["labels_list"] = df[LABEL_COL].apply(split_labels)

# 학습셋: 라벨이 있는 행만 사용
n_total = len(df)
df_trainable = df[df["labels_list"].map(len) > 0].copy()
n_dropped = n_total - len(df_trainable)
print(f"총 {n_total}행 중, 빈 라벨 {n_dropped}행 제외 → 학습 행 {len(df_trainable)}")

# 라벨 인코딩
all_labels = sorted({lab for labs in df_trainable["labels_list"] for lab in labs})
mlb = MultiLabelBinarizer(classes=all_labels)
Y = mlb.fit_transform(df_trainable["labels_list"])


from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
# Y: shape (n_samples, n_labels), dtype=int/uint8
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
idx_tr, idx_va = next(msss.split(np.arange(len(df_trainable)), Y))

df_tr = df_trainable.iloc[idx_tr].reset_index(drop=True)
df_va = df_trainable.iloc[idx_va].reset_index(drop=True)
Y_tr, Y_va = Y[idx_tr], Y[idx_va]
print("라벨 개수:", len(all_labels))
print("train:", df_tr.shape, "valid:", df_va.shape)



# 3) HF datasets 변환 & 토크나이즈

from datasets import Dataset
from transformers import AutoTokenizer

USE_TEXT_COL = TEXT_COL

train_ds = Dataset.from_dict({
    USE_TEXT_COL: df_tr[USE_TEXT_COL].astype(str).tolist(),
    "labels": Y_tr.astype("float32").tolist()
})
valid_ds = Dataset.from_dict({
    USE_TEXT_COL: df_va[USE_TEXT_COL].astype(str).tolist(),
    "labels": Y_va.astype("float32").tolist()
})

tok = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tok(batch[USE_TEXT_COL], truncation=True, padding="max_length", max_length=256)

train_ds = train_ds.map(tokenize, batched=True)
valid_ds = valid_ds.map(tokenize, batched=True)

for ds in (train_ds, valid_ds):
    ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

"tokenized"



# 4) 모델 정의 & 학습

import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, jaccard_score

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(all_labels),
    problem_type="multi_label_classification"
)

def compute_metrics(pred):
    logits, labels = pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    micro = f1_score(labels, preds, average="micro", zero_division=0)
    macro = f1_score(labels, preds, average="macro", zero_division=0)
    jacc = jaccard_score(labels, preds, average="samples")
    return {"micro_f1": micro, "macro_f1": macro, "jaccard": jacc}

args = TrainingArguments(
    OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,          # (공간 절약 권장)
    save_only_model=True,        # (옵티마이저 저장 안 함: 디스크 에러 방지)
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=10,           # 필요시 4~5로 증가
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    greater_is_better=True,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    bf16=True if torch.cuda.is_available() else False
)


class WeightedBCETrainer(Trainer):
    def __init__(self, pos_weight=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # pos_weight는 라벨 불균형 보정용
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        if self.pos_weight is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss(
                pos_weight=self.pos_weight.to(logits.device)
            )
        else:
            loss_fct = torch.nn.BCEWithLogitsLoss()

        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss
        
# Y: MultiLabelBinarizer 결과 (train 라벨 매트릭스)
label_freq = Y_tr.sum(axis=0)
pos_weight = torch.tensor((len(Y_tr) - label_freq) / (label_freq + 1e-6), dtype=torch.float)

trainer = WeightedBCETrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tok,
    compute_metrics=compute_metrics,
    pos_weight=pos_weight     # 불균형 보정 (없으면 None)
)

trainer.train()
trainer.save_model("hf_xlmr_ckpt")   # 모델 가중치 + config.json
tok.save_pretrained("hf_xlmr_ckpt") # 토크나이저 관련 파일까지 같이 저장


# 5) 체크포인트에 라벨/메타 저장


import os, json
os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(os.path.join(OUTPUT_DIR, "labels_ko.json"), "w", encoding="utf-8") as f:
    json.dump(all_labels, f, ensure_ascii=False, indent=2)
with open(os.path.join(OUTPUT_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump({"text_col": USE_TEXT_COL}, f, ensure_ascii=False, indent=2)
"saved labels/meta"



# 6) 라벨별 임계값 최적화
from sklearn.metrics import f1_score
import numpy as np, os

pred = trainer.predict(valid_ds)
probs = 1 / (1 + np.exp(-pred.predictions))   # (N,L)
labels = pred.label_ids                        # (N,L)


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
import numpy as np

# ---------------------------
# 1) Transformer 확률
pred = trainer.predict(valid_ds)
transformer_probs = 1 / (1 + np.exp(-pred.predictions))
labels = pred.label_ids

# ---------------------------
# 2) XGBoost (TF-IDF 기반)
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X_tr_tfidf = vectorizer.fit_transform(df_tr[TEXT_COL].astype(str))
X_va_tfidf = vectorizer.transform(df_va[TEXT_COL].astype(str))

xgb = OneVsRestClassifier(
    XGBClassifier(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        tree_method="gpu_hist" if torch.cuda.is_available() else "auto"
    ),
    n_jobs=-1
)
xgb.fit(X_tr_tfidf, Y_tr)
xgb_probs = np.array(xgb.predict_proba(X_va_tfidf))

# ---------------------------
# 3) Stacking 입력 만들기 (Transformer + XGB)
stack_train = np.hstack([transformer_probs, xgb_probs])  # (N, 2L)

# 4) 메타 모델 정의 (LogisticRegression 추천)
meta_model = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, solver="liblinear")
)
meta_model.fit(stack_train, labels)

# ---------------------------
# 5) 평가
stack_preds = meta_model.predict(stack_train)
print("== STACKING ==")
print("Micro-F1:", f1_score(labels, stack_preds, average="micro"))
print("Macro-F1:", f1_score(labels, stack_preds, average="macro"))
print(classification_report(labels, stack_preds, target_names=all_labels, zero_division=0))

ths = np.arange(0.30, 0.71, 0.05)
best_th_per_label = np.zeros(probs.shape[1], dtype=float) + 0.5

for j in range(probs.shape[1]):
    best_f1, best_th = -1, 0.5
    for th in ths:
        y_pred = (probs[:, j] >= th).astype(int)
        f1 = f1_score(labels[:, j], y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1, best_th = f1, th
    best_th_per_label[j] = best_th

np.save(os.path.join(OUTPUT_DIR, "label_thresholds.npy"), best_th_per_label)
best_th_per_label[:10], float(best_th_per_label.mean())



#  7) 추론(한국어 문장도 바로)

import torch, numpy as np, json, os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tok = AutoTokenizer.from_pretrained("hf_xlmr_ckpt")
mdl = AutoModelForSequenceClassification.from_pretrained("hf_xlmr_ckpt")


def load_model_for_infer(ckpt_dir=OUTPUT_DIR):
    tok_i = AutoTokenizer.from_pretrained(ckpt_dir)
    mdl_i = AutoModelForSequenceClassification.from_pretrained(ckpt_dir)
    labels_ko = json.load(open(os.path.join(ckpt_dir, "labels_ko.json"), "r", encoding="utf-8"))
    th_path = os.path.join(ckpt_dir, "label_thresholds.npy")
    per_label_th = np.load(th_path) if os.path.exists(th_path) else None
    meta = json.load(open(os.path.join(ckpt_dir, "meta.json"), "r", encoding="utf-8"))
    return tok_i, mdl_i, labels_ko, per_label_th, meta

tok_i, mdl_i, labels_ko, per_label_th, meta = load_model_for_infer(OUTPUT_DIR)

def predict_accords(texts, use_per_label_threshold=True, base_threshold=0.5, topk=10):
    if isinstance(texts, str):
        texts = [texts]
    inputs = tok_i(texts, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        logits = mdl_i(**inputs).logits
    probs = torch.sigmoid(logits).cpu().numpy()

    outs = []
    for i, p in enumerate(probs):
        if use_per_label_threshold and per_label_th is not None:
            mask = (p >= per_label_th)
        else:
            mask = (p >= base_threshold)
        picked = [(labels_ko[j], float(p[j])) for j in np.where(mask)[0]]
        top = sorted([(labels_ko[j], float(p[j])) for j in range(len(labels_ko))], key=lambda x: -x[1])[:topk]
        outs.append({"text": texts[i], "picked": picked, "topk": top})
    return outs

# 예시: 한국어 문장
ex = "달콤한 바닐라에 따뜻한 스파이스가 포근하게 감싸는 겨울 향"
res = predict_accords(ex, use_per_label_threshold=True)
res[0]


# 8) (선택) 검증셋 지표 + 라벨별 리포트

from sklearn.metrics import classification_report

# 0.5 기준 간단 리포트
pred = trainer.predict(valid_ds)
probs = 1 / (1 + np.exp(-pred.predictions))
preds = (probs >= 0.5).astype(int)
print("== threshold 0.5 ==")
print(classification_report(pred.label_ids, preds, target_names=all_labels, zero_division=0))

# per-label threshold 리포트 (있을 때)
if per_label_th is not None:
    preds2 = (probs >= per_label_th).astype(int)
    print("== per-label thresholds ==")
    print(classification_report(pred.label_ids, preds2, target_names=all_labels, zero_division=0))


총 26319행 중, 빈 라벨 264행 제외 → 학습 행 26055
라벨 개수: 17
train: (23442, 7) valid: (2613, 7)


Map:   0%|          | 0/23442 [00:00<?, ? examples/s]

Map:   0%|          | 0/2613 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_491/2627585995.py:126: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedBCETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1,Jaccard
1,1.604900,1.132991,0.457167,0.243429,0.332841
2,1.095200,1.094847,0.486406,0.294273,0.371129
3,0.850400,1.117922,0.490495,0.323341,0.385143
4,0.837600,1.067137,0.499012,0.324253,0.396898
5,1.141500,1.134249,0.522996,0.315478,0.423388
6,0.829500,1.247823,0.511377,0.333285,0.412796
7,0.651500,1.426886,0.515813,0.325188,0.419342
8,0.629500,1.603217,0.518010,0.319168,0.424824
9,0.428600,1.637656,0.520418,0.317388,0.428611
10,0.467700,1.690989,0.521035,0.308017,0.427979


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
!pip install xgboost